In [ ]:
import math

from data_util.db_util import *
import pandas as pd
import datetime

now = datetime.datetime.now()
date = now.strftime('%m%d')
dateindex = now.strftime('%Y-%m-%d')
print(dateindex, date)
file = '../temp/Table{}.xls'.format(date)


In [ ]:
from data_util.db_util import *
import pandas as pd
import datetime
file = '../temp/Table0111.xls'

df = pd.read_csv(file, encoding='gbk', sep='\t')
df['涨幅'] = df['涨幅'].str.strip('%')
df['5日涨幅'] = df['5日涨幅'].str.strip('%')
df['换手'] = df['换手'].str.strip('%')
print(df.columns)




In [ ]:
#treemap
import math
import sqlite3
import time

import pandas as pd

from pylab import mpl

from data_util.helper import read_ths_xlsx_to_df

mpl.rcParams['font.sans-serif'] = ['SimHei']  # 指定默认字体
mpl.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题



def mapcolor(v):
    # v = v * 100
    rg = [-4, -3, -2, -1, -0.5, -0.05, 0.05, 0.5, 1, 2, 3, 4]
    colors = ['#00d641', '#1aa448', '#0e6f2f', '#085421', '#083721', '#082221', '#424453', '#4d1414', '#6d1414', '#961010', '#be0808', '#e41414']

    for r in rg:
        if v < r:
            return colors[rg.index(r)]
    return colors[-1]


def mapcolors():
    # colors = ['#00d641', '#1aa448', '#0e6f2f', '#085421', '#424453', '#6d1414', '#961010', '#be0808', '#e41414']
    colors = ['#00d641', '#1aa448', '#0e6f2f', '#085421', '#083721', '#082221', '#424453', '#4d1414', '#6d1414', '#961010', '#be0808', '#e41414']

    ret = {}
    for c in colors:
        ret[c] = c
    return ret
    pass


df = pd.read_csv('../temp/Table0110.xls', encoding='gbk', sep='\t')
df['涨幅'] = df['涨幅'].str.strip('%')
df['换手'] = df['换手'].str.strip('%')
df['5日涨幅'] = df['5日涨幅'].str.strip('%')

df.to_pickle('temp/test.pkl')
#df = pd.read_pickle('temp/test.pkl')

df = df[['    名称', '总市值', '所属行业', '涨幅', '5日涨幅']]

df['总市值'] = pd.to_numeric(df['总市值'], errors='coerce')
df['涨幅'] = pd.to_numeric(df['涨幅'], errors='coerce')
df['5日涨幅'] = pd.to_numeric(df['5日涨幅'], errors='coerce')
df['总市值'] = round(df['总市值'] / 1e8, 1)
df['涨幅'] = round(df['5日涨幅'], 1)



In [ ]:
df = df[df['总市值'] > 1]

df['all'] = 'all'
df['color'] = df['涨幅'].map(mapcolor)

print(df)

import plotly.express as px
mcolors = mapcolors()
mcolors['(?)'] = '#999999'

fig = px.treemap(df,
                 path=['all', '所属行业', '    名称'],
                 # labels='涨幅',
                 # names='    名称',
                 values='总市值',
                 color='color',
                 custom_data=['涨幅'],
                 color_discrete_map=mcolors,
                 hover_data=['涨幅'],
                 )
fig.update_traces(root_color="lightgray",
                  texttemplate='<b>%{label}</b><br><b>%{value:.2f}</b><br><b>%{customdata[0]:.2f}</b><br>',
                  )
fig.update_layout(margin=dict(t=1, l=1, r=1, b=1))
img = fig.to_image('png')
with open('temp/test.png', 'wb') as fs:
    fs.write(img)
fig.show()

In [ ]:
# 更新db 数据

def ts_code(code):
    if len(code) == 8:
        return f'{code[2:]}.{code[:2]}'
    else:
        return ''

conn = sqlite3.connect('../stocks.db')


for i,row in df.iterrows():
    code = row.loc['代码']
    bk = row.loc['细分行业']
    pe = row.loc['TTM市盈率']
    pb = row.loc['市净率']
    # print(code, bk, pe, pb)
    if math.isnan(pe):
        pe = 0
    if math.isnan(pb):
        pb = 0
    tc = ts_code(code)
    if len(tc)==9:
        conn.execute('''update STOCKBASIC set pe={}, pb={}, bk='{}' where ts_code = '{}' '''.format(pe,pb,bk, tc))
        # conn.commit()
        # print(f'update {i} {tc} {pe} {pb} {bk}')
    # break
conn.commit()
conn.close()
print('update finish')



In [ ]:
# 查询板块

conn = sqlite3.connect('../stocks.db')

bks = conn.execute('''select distinct bk from STOCKBASIC''').fetchall()

print(len(bks))
for bk in bks:
    print(bk)

conn.close()



In [2]:
# 查看市值分布
import sqlite3

def select_mv_count(mv):
    conn = sqlite3.connect('../stocks.db')
    ret = conn.execute('''select count(*) from STOCKBASIC where marketvalue > {}'''.format(mv)).fetchall()
    return ret[0][0]



mv = [10000,2000, 1000, 500, 100, 50, 10, 0]

# 统计各种市值的数量
mvcount = [select_mv_count(v) for v in mv]
with open('temp/day_marketvalue.csv', 'w') as fs:
    fs.write('marketvalue, count\n')

    for i in range(len(mv)):
        mvi = mv[i]
        mvcounti = mvcount[i]
        if i == 0:
            print('mv > {} count {}'.format(mvi, mvcounti))
        if i > 0:
            print('{} > mv > {} count {}'.format(mv[i - 1], mvi, mvcounti - mvcount[i - 1]))
        fs.write('{},{}\n'.format(mvi, mvcounti))
        pass


mv > 10000 count 7
10000 > mv > 2000 count 57
2000 > mv > 1000 count 99
1000 > mv > 500 count 151
500 > mv > 100 count 1319
100 > mv > 50 count 1188
50 > mv > 10 count 1791
10 > mv > 0 count 3


In [ ]:
import matplotlib.pyplot as plt

#plt.bar(mv, mvcount)
plt.plot(range(len(mvcount)), mvcount)
plt.grid()
plt.show()









